In [ ]:
!pip install karateclub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 1.8 MB 20.3 MB/s 
     |████████████████████████████████| 24.1 MB 109.2 MB/s 
     |████████████████████████████████| 170 kB 60.2 MB/s 
     |████████████████████████████████| 2.2 MB 44.7 MB/s 
  Created wheel for karateclub: filename=karateclub-1.3.2-py3-none-any.whl size=99091 sha256=f33eb99e447830974375fcebd2318bcffc1d327b717b4a5e5638192063b8e786
  Stored in directory: /root/.cache/pip/wheels/6c/56/60/90b1d642c510370d800bb81d4883e92023c45e3b7ca19cf1af
Successfully built karateclub
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from karateclub.graph_embedding import Graph2Vec
import numpy as np
from tqdm import tqdm
import os
import glob
from pathlib import Path
import os.path

def get_nodes(filename):  
    #lists to get the parents and the children
    parents=[]
    children=[]
    
    f = open(filename, "r")
    for x in f:
        
        #print(x)
        #print(x.split(","))

        #the first node is the parent and the second one is the child
        if(x[0]=='\n'):
          continue
        x1 = x.split(",",1)
        #print(x1)
        parent,child=x1[0],x1[1]
        #parent,child=x.split(",")[0],x.split(",")[1]
        
        parent,child = x1[0],x1[1]

        #appending the nodes to the respective list
        parents.append(parent)
        children.append(child[:-1])
    #
    nodes = list(set(set(parents).union(set(children))))
    #for i in nodes:
      #print(i)
    return [parents,children,nodes]


def get_node_labels(nodes,parents,children):
    #
    labels_list=list()
    for i in nodes:
        i1 = i.split("--")[0]
        #print(i)
        labels_list.append(int(i1))
    labels_list.sort()

    label_keys=dict()
    for i in range(len(labels_list)):
        label_keys[labels_list[i]]=i

    labels=dict()
    for i in nodes:
        i1=i.split("--")
        key=label_keys[int(i1[0])]
        labels[key]=i1[1]

    sorted_labels = sorted(labels.items(), key=lambda x: x[0])
    
    edges = []
    for i,j in zip(parents,children):
        parent=i.split("--")[0]
        child=j.split("--")[0]
        edges.append((label_keys[int(parent)],label_keys[int(child)]))
    edges.sort()

    label=dict()
    for i in sorted_labels:
        label[i[0]]=i[1]
    #print(label)
    #print(edges)
    return [label,edges]

def graph(edges,label):
    #print(len(edges))
    #print(edges)
    tree2graph = nx.DiGraph()
    tree2graph.add_edges_from(edges)
    for node in tree2graph.nodes():
        tree2graph.nodes[node]['feature'] = label[node]
    return tree2graph

def get_graph(file):
    l=get_nodes(file)
    parents,children,nodes = l[0],l[1],l[2] 
    l=get_node_labels(nodes,parents,children)
    #print(l)
    label,edges=l[0],l[1] 
    g=graph(edges,label)
    return g

# Graph2Vec attributed example
def get_embedding(graphs):
    model = Graph2Vec(attributed=True)
    model.fit(graphs)
    embed = model.get_embedding()
    return embed


def getParent(path, levels = 1):
    common = path
    for i in range(levels + 1):
        common = os.path.dirname(common)
    return os.path.relpath(path, common)


if __name__ == '__main__':
    directory = r'/content/drive/MyDrive/Java_Nodes'    # Base Directory
    print(directory)
    lang = ['C','Python','Java']
    directories = ['/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes']
    #for i in lang:
    #    directories.append(directory + '\\' + i + "_Nodes")

    x = []
    y = []
    total = []
    classes = set()

    for directory in tqdm(list(directories)):
        for root, subdirectories, files in tqdm(list(os.walk(directory))):
            for subdirectory in subdirectories:
                classes.add(subdirectory)
            for file in tqdm(list(files)):
                path = os.path.join(root, file)

                total.append(path)

                cmplx_class = getParent(path,).split('\\')[0]
                node_lang = getParent(path,2).split('\\')[0]
                fname = Path(path).stem

                if node_lang == 'Python_Nodes':
                    file_type = 'Python'
                elif node_lang == 'C_Nodes':
                    file_type = 'C'
                else:
                    file_type = 'Java'
                
                entry = (cmplx_class,fname,file_type)
                y.append(entry)
    
    # print(total)      # Paths of all files
    # print(classes)    # Total Complexity Classes
    # print(y)          # list of Tuples -> (Complexity Class, file name, Lang)

    graphs = []
    
    # total = [
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes0.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes1.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes2.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes3.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes4.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes5.txt',
    # r'C:\Users\nikhi\OneDrive\Documents\gitUploads\Capstone_Dataset\Java_Nodes\O(logN)\nodes6.txt']
    files = []
    for i in total:
        print("--------")
        print(i)
        files.append(i)
        g = get_graph(i)
        graphs.append(g)


/content/drive/MyDrive/Java_Nodes


  0%|          | 0/11 [00:00<?, ?it/s]

0it [00:00, ?it/s]


100%|██████████| 26/26 [00:00<00:00, 9197.26it/s]


100%|██████████| 3/3 [00:00<00:00, 6574.14it/s]


100%|██████████| 6/6 [00:00<00:00, 7403.89it/s]

 36%|███▋      | 4/11 [00:00<00:00, 38.53it/s]

100%|██████████| 11/11 [00:00<00:00, 9601.94it/s]


100%|██████████| 1/1 [00:00<00:00, 3379.78it/s]


100%|██████████| 1/1 [00:00<00:00, 3182.32it/s]


100%|██████████| 7/7 [00:00<00:00, 8004.40it/s]

 73%|███████▎  | 8/11 [00:00<00:00, 36.66it/s]

100%|██████████| 61/61 [00:00<00:00, 5721.72it/s]


100%|██████████| 1/1 [00:00<00:00, 3443.60it/s]


100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes0.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes3.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes1.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes2.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes5.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes6.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes4.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes8.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes7.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes9.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes11.txt
--------
/content/drive/MyDrive/Capstone_Dataset-main/C_Nodes/O(N2)/nodes12.txt
--------
/content/drive/MyDrive/Capstone_Dataset-m

In [ ]:
embeddings = []
embeddings=get_embedding(graphs)
    
for i in tqdm(list(graphs)):
        print(i.nodes.data())
        print('\n\n')

100%|██████████| 119/119 [00:00<00:00, 2490.99it/s]

[(0, {'feature': 'FileAST'}), (1, {'feature': 'FuncDef'}), (2, {'feature': 'FuncDef'}), (3, {'feature': 'FuncDef'}), (4, {'feature': 'Decl'}), (6, {'feature': 'Compound'}), (7, {'feature': 'Compound'}), (8, {'feature': 'Decl'}), (5, {'feature': 'Decl'}), (9, {'feature': 'Compound'}), (10, {'feature': 'FuncDecl'}), (13, {'feature': 'FuncDecl'}), (14, {'feature': 'Return'}), (11, {'feature': 'FuncCall'}), (15, {'feature': 'Decl'}), (16, {'feature': 'Return'}), (17, {'feature': 'For'}), (12, {'feature': 'FuncDecl'}), (18, {'feature': 'Return'}), (19, {'feature': 'FuncCall'}), (20, {'feature': 'Decl'}), (21, {'feature': 'Decl'}), (22, {'feature': 'ParamList'}), (24, {'feature': 'comp'}), (25, {'feature': 'qsort'}), (26, {'feature': 'ExprList'}), (23, {'feature': 'findNumberOfTriangles'}), (27, {'feature': 'ParamList'}), (29, {'feature': 'main'}), (30, {'feature': '>'}), (31, {'feature': '0'}), (32, {'feature': 'count'}), (33, {'feature': 'count'}), (34, {'feature': '++'}), (35, {'feature':

In [ ]:
    """Returns the minimum length of a
    run from 23 - 64 so that
    the len(array)/minrun is less than or
    equal to a power of 2.
 
    e.g. 1=>1, ..., 63=>63, 64=>32, 65=>33,
    ..., 127=>64, 128=>32, ...
    """

'Returns the minimum length of a\nrun from 23 - 64 so that\nthe len(array)/minrun is less than or\nequal to a power of 2.\n \ne.g. 1=>1, ..., 63=>63, 64=>32, 65=>33,\n..., 127=>64, 128=>32, ...\n'

In [ ]:
embeddings[0].shape

(128,)

In [ ]:
embeddings[0]

array([ 0.44880074,  0.05052464, -0.1818467 ,  0.03988037, -0.244624  ,
        0.00678686,  0.1527714 , -0.05742872,  0.02933129, -0.00761087,
        0.0826095 ,  0.21860836, -0.2057446 ,  0.32485485,  0.10289663,
        0.09630104,  0.10503216, -0.10492171, -0.13004579,  0.09767567,
       -0.08718912, -0.23188914,  0.07550967, -0.00599401, -0.05791152,
       -0.14705698,  0.3630431 , -0.09131167, -0.16174619, -0.20727667,
       -0.14187896,  0.00923686,  0.14854535,  0.02506721, -0.20076449,
        0.07143384,  0.06378492,  0.09913768,  0.37172577,  0.04998683,
       -0.15933698,  0.00291997, -0.10225929,  0.3399757 ,  0.07036085,
       -0.09526601,  0.11413297, -0.0979269 ,  0.03295238, -0.05791393,
        0.23559096,  0.14238898,  0.16498543,  0.16924845,  0.16571553,
        0.06861541,  0.0916013 ,  0.03920014, -0.30830184,  0.2669437 ,
       -0.05927255,  0.06552795,  0.35509732,  0.14372453,  0.19581218,
       -0.09582803, -0.09442866,  0.05598057,  0.06907187, -0.05